In [10]:
import pandas as pd

## 전체 train sample
df = pd.read_csv('../train.csv',index_col=0)
set_level_list_3 = list(set(list(df['cat3'])))
set_level_list_3.sort()            
label_2_number = {label:i for i,label in enumerate(set_level_list_3)}
number_2_label = {i:label for i,label in enumerate(set_level_list_3)}

In [14]:
label_2_number

{'5일장': 0,
 'ATV': 1,
 'MTB': 2,
 '강': 3,
 '게스트하우스': 4,
 '계곡': 5,
 '고궁': 6,
 '고택': 7,
 '골프': 8,
 '공연장': 9,
 '공예,공방': 10,
 '공원': 11,
 '관광단지': 12,
 '국립공원': 13,
 '군립공원': 14,
 '기념관': 15,
 '기념탑/기념비/전망대': 16,
 '기암괴석': 17,
 '기타': 18,
 '기타행사': 19,
 '농.산.어촌 체험': 20,
 '다리/대교': 21,
 '대중콘서트': 22,
 '대형서점': 23,
 '도립공원': 24,
 '도서관': 25,
 '동굴': 26,
 '동상': 27,
 '등대': 28,
 '래프팅': 29,
 '면세점': 30,
 '모텔': 31,
 '문': 32,
 '문화관광축제': 33,
 '문화원': 34,
 '문화전수시설': 35,
 '뮤지컬': 36,
 '미술관/화랑': 37,
 '민물낚시': 38,
 '민박': 39,
 '민속마을': 40,
 '바/까페': 41,
 '바다낚시': 42,
 '박람회': 43,
 '박물관': 44,
 '발전소': 45,
 '백화점': 46,
 '번지점프': 47,
 '복합 레포츠': 48,
 '분수': 49,
 '빙벽등반': 50,
 '사격장': 51,
 '사찰': 52,
 '산': 53,
 '상설시장': 54,
 '생가': 55,
 '서비스드레지던스': 56,
 '서양식': 57,
 '섬': 58,
 '성': 59,
 '수련시설': 60,
 '수목원': 61,
 '수상레포츠': 62,
 '수영': 63,
 '스노쿨링/스킨스쿠버다이빙': 64,
 '스카이다이빙': 65,
 '스케이트': 66,
 '스키(보드) 렌탈샵': 67,
 '스키/스노보드': 68,
 '승마': 69,
 '식음료': 70,
 '썰매장': 71,
 '안보관광': 72,
 '야영장,오토캠핑장': 73,
 '약수터': 74,
 '연극': 75,
 '영화관': 76,
 '온천/욕장/스파': 77,
 '외국문화원

In [11]:
## 베스트 모델 추론할때 soft 보팅의 웨이트값을 찾기 위함
result_df = pd.DataFrame(list(label_2_number.keys()),columns=['index'])

target_df = pd.read_csv('./data_save_temp/fold_3_epoch_15_valdf_rank_0.csv')

target_df = target_df[['id','cat3','predict_label_number','target','predict_correct']]
df1 = target_df['cat3'].value_counts().to_frame(name='val_전체수').reset_index()
df2 = target_df.groupby(['cat3'])['predict_correct'].sum().to_frame(name='val_정답수').reset_index()
result_df = pd.merge(left = result_df , right = df1, left_on = "index",right_on = "index", how = "left")
result_df


FileNotFoundError: [Errno 2] No such file or directory: './data_save_temp/fold_3_epoch_15_valdf_rank_0.csv'

In [ ]:
result_df = pd.merge(left = result_df , right = df2, left_on = "index",right_on = "cat3", how = "left").drop('cat3',1)

result_df['val_틀린수'] = result_df['val_전체수'] - result_df['val_정답수']
result_df['acc'] = result_df['val_정답수'] / result_df['val_전체수']
result_df = result_df.fillna(0)
result_df
# result_df.to_csv('./temp.csv')


In [ ]:
weight_dict = { index:acc for index,acc in zip(result_df['index'],result_df['acc'])}
weight_dict

In [ ]:
target_df = pd.read_csv('./data_save_temp/fold_3_epoch_15_valdf_rank_0.csv')
temp = target_df.iloc[0]['predict_logit'].replace('[','')
temp = temp.replace(']','')
temp = temp.replace("'",'')
temp = temp.replace(" ",'')
temp = temp.split(',')
print(temp,type(temp))

In [ ]:
import numpy as np
temp2 = []
for i in temp:
    temp2.append(float(i))
    
print(np.max(temp2),np.argmax(temp2))

In [ ]:
print(number_2_label[103],target_df.iloc[0])

In [ ]:
from numpy import dtype
import torch
t_temp = torch.Tensor(temp2)
print(t_temp)
s_temp = torch.nn.functional.softmax(t_temp,dim=0)
print(s_temp)

In [ ]:
torch.max(s_temp)

In [ ]:
from scipy.special import softmax
print(softmax(temp2))

In [ ]:
import numpy as np

temp = np.array([[0,1,2],[4,5,6],[9,-1,2]])
temp2 = np.array([0.5,0.5,0.5])

print(temp*temp2)

In [25]:
from cv2 import merge
import pandas as pd

df1 = pd.read_csv('./soft_voting_large_best_f1.csv')
df2 = pd.read_csv('./valdf작업.csv')
df3 = pd.read_csv('./soft_voting_large_weght_best_f1.csv')
df4 = pd.read_csv('./soft_voting_large_best_valloss_token17.csv')

df1 = df1[['id','soft_voitng']]
df2 = df2[['id','my_labeling']]
df3 = df3[['id','soft_voitng']]
df4 = df4[['id','soft_voitng']]

diff_df = pd.merge(left=df1,right=df2,left_on='id',right_on='id',how="left")
# diff_df = pd.merge(left=diff_df,right=df3,left_on='id',right_on='id',how="left")
# diff_df = pd.merge(left=diff_df,right=df4,left_on='id',right_on='id',how="left")

diff_df['diff'] = diff_df.apply(lambda x:x['soft_voitng']==x['my_labeling'],axis=1)
# diff_df['diff'] = diff_df.apply(lambda x:x['soft_voitng_x']==x['soft_voitng_y'] and x['soft_voitng_x']==x['soft_voitng'],axis=1)
diff_df.to_csv('./total.csv')

In [26]:
diff_df

,id,soft_voitng,my_labeling,diff
0,TEST_00000,한식,한식,True
1,TEST_00001,한식,한식,True
2,TEST_00002,한식,한식,True
3,TEST_00003,수련시설,수련시설,True
4,TEST_00004,산,산,True
...,...,...,...,...
7275,TEST_07275,한식,한식,True
7276,TEST_07276,공연장,공연장,True
7277,TEST_07277,"야영장,오토캠핑장","야영장,오토캠핑장",True
7278,TEST_07278,모텔,모텔,True


In [27]:
diff_df.groupby('diff').size()

diff
False     190
True     7090
dtype: int64

In [28]:
diff_df[diff_df['diff']==False]
# diff_df[diff_df['diff']==False].to_csv('./diff.csv')

,id,soft_voitng,my_labeling,diff
27,TEST_00027,한식,이색거리,False
77,TEST_00077,자연생태관광지,다리/대교,False
96,TEST_00096,이색거리,농.산.어촌 체험,False
106,TEST_00106,기암괴석,산,False
300,TEST_00300,일반축제,박람회,False
...,...,...,...,...
7147,TEST_07147,한식,일식,False
7169,TEST_07169,도립공원,군립공원,False
7175,TEST_07175,전시관,박물관,False
7199,TEST_07199,이색거리,자연생태관광지,False
